In [31]:
import importlib
import pickle
import constants_and_methods
importlib.reload(constants_and_methods)
from constants_and_methods import Card, RANKS, SUITS, evaluate_hand, expand_evaluate_hand, get_hand_rep, get_order_score

In [32]:
with open('data/raw_split_data.pkl', 'rb') as file:
    raw_data = pickle.load(file)
raw_data_size = len(raw_data['_treatments'])

In [33]:
raw_data.keys()

dict_keys(['_treatments', '_outcomes', '_profits', '_raiser_caller_positions', '_open_sizes', '_raisers_stack', '_callers_stack', '_pot_sizes', '_flops_cards', '_hero_hands'])

In [34]:
raw_data_size

939

# prepare Data

### flop & hero hand
create a one-hot representation
- hero hand is mapped into 196 possible categories (e.g. KdQs -> KQo)
- hero hand + flop cards are mapped into 9 possible hand strengths (e.g. full house)
- flop cards are mapped into 3 possible hand strengths (e.g. two pair)
- haro hand + flop cards are mapped into 16 possible draws (e.g. flush draw)
- flop cards are mapped into 4 possible draws
- additional vector represent value order between hero hand and flop cards with 10 categories

In [35]:
suit_mapper = {'c': '♣', 'd': '♦', 'h': '♥', 's': '♠'}
num_mapper = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}

convert cards into Card instance representation

In [36]:
flops_cards = []
for flop in raw_data['_flops_cards']:
    flop_rep = []
    for str_card in flop:
        int_val = num_mapper[str_card[0]]
        suit_val = suit_mapper[str_card[1]]
        flop_rep.append(Card(int_val, suit_val))
    flops_cards.append(flop_rep)

hero_hands = []
for hand in raw_data['_hero_hands']:
    fix_hand = [hand[1:3], hand[4:6]]
    hand_rep = []
    for card in fix_hand:
        int_val = num_mapper[card[0]]
        suit_val = suit_mapper[card[1]]
        hand_rep.append(Card(int_val, suit_val))
    hero_hands.append(hand_rep)

map Hero hand into hash value

In [37]:
hands = []
for hh in hero_hands:
    hands.append(get_hand_rep(hh))

map draws and potentials into hash value

In [38]:
def draws_to_int(draws_bools):
    binary_list = [0 if v is False else 1 for v in draws_bools[::-1]]
    binary_string = ''.join(map(str, binary_list))  # Converts list to "0110"
    binary_value = int(binary_string, 2)  # Converts "0110" to integer 6
    return binary_value

In [39]:
hands_strength = []  # 0 to 9
flops_strength = []  # 0 to 2
hands_potential = []  # 0 to 15
flops_potential = []  # 0 to 3

for hh, f in zip(hero_hands, flops_cards):
    hands_strength.append(evaluate_hand(hh+f).hand_rank)
    flops_strength.append(evaluate_hand(f).hand_rank)
    hands_potential.append(draws_to_int(expand_evaluate_hand(hh+f)))
    flops_potential.append(draws_to_int(expand_evaluate_hand(f)))

map order between hero hand and flop cards into hash value

In [40]:
cards_order = []
for hh, f in zip(hero_hands, flops_cards):
    cards_order.append(get_order_score(f, hh))

### all other

In [41]:
treatment = raw_data['_treatments']
outcomes = [0 if v == 'fold' else (1 if v == 'call' else 2) for v in raw_data['_outcomes']]  # label
profits = raw_data['_profits']
raiser_position = [v[0] for v in raw_data['_raiser_caller_positions']]  # to embedd (in [0,5[)
caller_position = [v[1] for v in raw_data['_raiser_caller_positions']]  # to embedd (in [0,5])
open_sizes = [v for v in raw_data['_open_sizes']]
raiser_stack = [v for v in raw_data['_raisers_stack']]
caller_stack = [v for v in raw_data['_callers_stack']]
pot_size = [v for v in raw_data['_pot_sizes']]
checked_to = [int(rp > cp) for rp, cp in zip(raiser_position, caller_position)]

# Save

In [42]:
all_lists = [treatment, outcomes, raiser_position, caller_position, open_sizes, raiser_stack, caller_stack,
             pot_size, checked_to, hands, hands_strength, flops_strength, hands_potential, flops_potential,
             cards_order, profits]

In [43]:
with open('data/lists_data.pkl', 'wb') as file:
    pickle.dump(all_lists, file)